In [70]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.2 MB/s eta 0:00:00


In [71]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

In [34]:
train = pd.read_csv('/content/drive/MyDrive/train.csv', index_col=[0])
test = pd.read_csv('/content/drive/MyDrive/test.csv')

In [36]:
train.shape

(11504798, 11)

In [37]:
test.shape

(7669866, 11)

In [38]:
def label_encode_gender(val):
    if val == 'Female':
        return 1
    elif val == 'Male':
        return 0
    return None

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
train['Gender'] = train['Gender'].apply(label_encode_gender)
test['Gender'] = test['Gender'].apply(label_encode_gender)

In [41]:
def label_encode_vehicle_age(val):
    if val == '< 1 Year':
        return 0
    elif val == '1-2 Year':
        return 1
    elif val == '> 2 Years':
        return 2
    return None

In [42]:
train['Vehicle_Age'] = train['Vehicle_Age'].apply(label_encode_vehicle_age)
test['Vehicle_Age'] = test['Vehicle_Age'].apply(label_encode_vehicle_age)

In [43]:
def label_encode_vehicle_damage(val):
    if val == 'Yes':
        return 1
    else:
        return 0
    return None

In [44]:
train['Vehicle_Damage'] = train['Vehicle_Damage'].apply(label_encode_vehicle_damage)
test['Vehicle_Damage'] = test['Vehicle_Damage'].apply(label_encode_vehicle_damage)

In [45]:
y = train['Response']
X = train.drop(columns=['Response'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

0:	total: 3.92s	remaining: 6m 27s
10:	total: 29.1s	remaining: 3m 55s


In [62]:
# Make predictions for training and testing sets
y_pred_train_proba = bst.predict(X_train, num_iteration=bst.best_iteration)
y_pred_test_proba = bst.predict(X_test, num_iteration=bst.best_iteration)

# Evaluate the model
train_auc = roc_auc_score(y_train, y_pred_train_proba)
test_auc = roc_auc_score(y_test, y_pred_test_proba)

print(f'Train AUC = {train_auc}')
print(f'Test AUC = {test_auc}')

Train AUC = 0.8722446828632238
Test AUC = 0.871783891249331


In [65]:
final_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    objective='binary',
    metric='auc',
    num_leaves=31,
    learning_rate=0.05,
    feature_fraction=0.9,
    n_estimators=100
)

final_model.fit(X, y)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Number of positive: 1415059, number of negative: 10089739
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.726232 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 737
[LightGBM] [Info] Number of data points in the train set: 11504798, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.122997 -> initscore=-1.964348
[LightGBM] [Info] Start training from score -1.964348


LGBMClassifier(feature_fraction=0.9, learning_rate=0.05, metric='auc',
               objective='binary')

In [66]:
response_values = final_model.predict_proba(test.iloc[:, 1:])[:, 1]

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9


In [67]:
response_values

array([0.01900826, 0.38143469, 0.26178252, ..., 0.00125766, 0.50729252,
       0.00119482])

In [51]:
test['id'].values

array([11504798, 11504799, 11504800, ..., 19174661, 19174662, 19174663])

In [68]:
data = {
    'id': test['id'].values,
    'Response': response_values
}

df = pd.DataFrame(data)

df.to_csv('submission.csv', index=False)

In [69]:
df

,id,Response
0,11504798,0.019008
1,11504799,0.381435
2,11504800,0.261783
3,11504801,0.001195
4,11504802,0.033235
...,...,...
7669861,19174659,0.206772
7669862,19174660,0.001195
7669863,19174661,0.001258
7669864,19174662,0.507293


In [54]:
df.shape

(7669866, 2)